In [30]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco import Parameters


from nmf_grooming import (
    NeuromechflyGrooming,
    all_groom_dofs,
    load_grooming_data,
    plot_state_and_contacts,
)
from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [21]:
run_time = 1

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

In [22]:
target_num_steps = int(run_time / sim_params.timestep)


In [23]:
nmf = flygym.mujoco.NeuroMechFly(
    sim_params = sim_params,
    init_pose = "stretch",
    actuated_joints = actuated_joints,
    control = "position",
)

In [24]:
data_path = flygym.common.get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

data_block = np.zeros((len(actuated_joints), target_num_steps))

input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * sim_params.timestep
for i, dof in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[dof])

In [25]:
obs, info = nmf.reset()

legs = [f"{side}{pos}" for side in "LR" for pos in "FMH"]
dofs_per_leg = [
    "Coxa",
    "Coxa_roll",
    "Coxa_yaw",
    "Femur",
    "Femur_roll",
    "Tibia",
    "Tarsus1",
]


In [26]:
all_joints_pos = np.copy(data_block)

for i, data in enumerate(data_block[:,0]):
    if (i > 6 and i < 13) or (i > 27 and i < 35):
        all_joints_pos[i, :] = np.linspace(data, 0, target_num_steps)
    elif (i > 0 and i < 7) or (i > 20 and i < 28):
        all_joints_pos[i, :] = np.linspace(data, 0, target_num_steps)
    else:
        all_joints_pos[i, :] = np.linspace(data, data, target_num_steps)

print(all_joints_pos)


[[ 6.01047575e-01  6.01047575e-01  6.01047575e-01 ...  6.01047575e-01
   6.01047575e-01  6.01047575e-01]
 [ 8.14028257e-01  8.13946846e-01  8.13865435e-01 ...  1.62821934e-04
   8.14109668e-05  0.00000000e+00]
 [-7.02044809e-02 -7.01974597e-02 -7.01904386e-02 ... -1.40423004e-05
  -7.02115020e-06  0.00000000e+00]
 ...
 [ 2.55398350e-01  2.55398350e-01  2.55398350e-01 ...  2.55398350e-01
   2.55398350e-01  2.55398350e-01]
 [ 1.12100818e+00  1.12100818e+00  1.12100818e+00 ...  1.12100818e+00
   1.12100818e+00  1.12100818e+00]
 [-4.33544092e-01 -4.33544092e-01 -4.33544092e-01 ... -4.33544092e-01
  -4.33544092e-01 -4.33544092e-01]]


In [27]:
adhesion = np.zeros(len(legs))
for i in range(len(legs)):
    if legs[i].endswith("M"):
        adhesion[i] = 1
    elif legs[i].endswith("F"):
        adhesion[i] = 1
    elif legs[i].endswith("H"):
        adhesion[i] = 0


In [28]:

for i in range(target_num_steps):
    joint_pos = all_joints_pos[:, i]
    action = {"joints": joint_pos, "adhesion": adhesion}
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/stretch.mp4")

In [29]:
Video("./outputs/stretch.mp4")

In [55]:
run_time = 1

legs = [f"{side}{pos}" for side in "LR" for pos in "FMH"]
dofs_per_leg = [
    "Coxa",
    "Coxa_roll",
    "Coxa_yaw",
    "Femur",
    "Femur_roll",
    "Tibia",
    "Tarsus1",
]

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    draw_adhesion=True,
    draw_contacts=True,
)


data_path = flygym.common.get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs
target_num_steps = int(run_time / sim_params.timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))

input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * sim_params.timestep
for i, dof in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[dof])
print(data_block[0])

nmf = flygym.mujoco.NeuroMechFly(
    sim_params=sim_params,
    init_pose="stretch",
    actuated_joints=actuated_joints,
    control="position",
)
obs, info = nmf.reset()

for i in trange(target_num_steps):
    
    for i, leg in enumerate(legs):
    joint_pos = data_block[:, 0]
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()


nmf.save_video("./outputs/test.mp4")

[0.60104758 0.6011889  0.60133022 ... 0.13406378 0.13406378 0.13406378]
[0. 0. 0. ... 0. 0. 0.]


NameError: name 'swing_periods' is not defined

In [52]:
Video("./outputs/test.mp4")

In [ ]:
 

L_midleg_start = obs["joints"][0][2]
L_midleg_stretch = np.linspace(L_midleg_start, 0.0, target_num_steps)

R_midleg_start = obs["joints"][0][5]
R_midleg_stretch = np.linspace(R_midleg_start, 0.0, target_num_steps)
print(L_midleg_stretch)


foreleg_swing_ids = np.linspace(0.0, swing_periods["RF"][1], target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], 2*np.pi, target_num_steps)
hind_ids = np.zeros(target_num_steps)




adhesion_action = np.array(0.0 if leg.endswith("H") else 1.0 for leg in legs)








joints_angles = {}
    for leg in legs:
        if leg.endswith("M"):
            midleg_joint_angles = preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i])
            if leg.startswith("R"):
                midleg_joint_angles += R_midleg_stretch[i]
            elif leg.startswith("L"):
                midleg_joint_angles += L_midleg_stretch[i]
            joints_angles.extend(midleg_joint_angles)
        elif leg.endswith("H"):
            joints_angles.extend(preprogrammed_steps.get_joint_angles(leg, hind_ids[i]))
        else:
            joints_angles.extend(preprogrammed_steps.get_joint_angles(leg, foreleg_swing_ids[i]))
    
    all_joint_angles.append(joints_angles.copy())


    action = {"joints": np.array(joints_angles), "adhesion": adhesion_action}